####Replace function

In [1]:
from google.colab import drive
!pip install -U spacy
!python -m spacy download fr_core_news_sm
!pip install verbecc
!pip install pattern
!pip install PyMuPDF
drive.mount('/content/drive')

     |████████████████████████████████| 5.8 MB 30.6 MB/s 
     |████████████████████████████████| 456 kB 42.6 MB/s 
     |████████████████████████████████| 10.1 MB 60.8 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 621 kB 43.6 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 17.1 MB 74 kB/s 
✔ Download and installation successful
You can

     |████████████████████████████████| 22.2 MB 1.6 MB/s 
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 81 kB 9.0 MB/s 
     |████████████████████████████████| 5.6 MB 54.6 MB/s 
     |████████████████████████████████| 5.6 MB 44.3 MB/s 
     |████████████████████████████████| 419 kB 43.8 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 3.0 MB 41.4 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=b37a30753de15e9abdd0bbaa281c5921137bb802b1c58b37b3ca2bf4af1413d0
  Stored in directory: /root/.cache/pip/wheels/8d/1f/4e/9b67afd2430d55dee90bd57618dd7d899f1323e5852c465682
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp37-cp37m-linux_x86_64.whl size=100137 sha256=7584d211ffcb2e65fbfc73b269f7c012b6fd9783e2b753149b758a21243f0fd7
  Stored in directory: /root/.cache/pip/wheels/79/1c/f8/11fafab45fe6696eea63794a5d747b9c6b54990ac6f1885fb7
  C

In [2]:
# a quoi ressemble un fichier dico
import pandas as pd
dico_a = pd.read_csv('/content/drive/MyDrive/remplacement/dicts/dico-b.tsv', sep='\t', header=None, usecols = [0,1], names=["mot", "syn"])
dico_a.head()

,mot,syn
0,babeurre,lait battut
1,babiller,bavardage
2,babiole,jouets
3,babord,gauche
4,babouche,chaussure


In [3]:
#creer le dataframe de mot et synonime de tous les dico
import glob

all_files = glob.glob("/content/drive/MyDrive/remplacement/dicts/*.tsv")

li = []
for filename in all_files:
    print(filename)
    df = pd.read_csv(filename, sep='\t',header=None,   index_col=0,  usecols = [0,1], names=["mot", "syn"])
    li.append(df)

mot_syn = pd.concat(li)

mot_syn
# Question pourquoi vous avez choisit les mots comme indexes du DataFrame?


/content/drive/MyDrive/remplacement/dicts/dico-i.tsv
/content/drive/MyDrive/remplacement/dicts/dico-a.tsv
/content/drive/MyDrive/remplacement/dicts/dico-d.tsv
/content/drive/MyDrive/remplacement/dicts/dico-b.tsv
/content/drive/MyDrive/remplacement/dicts/dico-e.tsv
/content/drive/MyDrive/remplacement/dicts/dico-z.tsv
/content/drive/MyDrive/remplacement/dicts/dico-y.tsv
/content/drive/MyDrive/remplacement/dicts/dico-x.tsv
/content/drive/MyDrive/remplacement/dicts/dico-w.tsv
/content/drive/MyDrive/remplacement/dicts/dico-v.tsv
/content/drive/MyDrive/remplacement/dicts/dico-u.tsv
/content/drive/MyDrive/remplacement/dicts/dico-t.tsv
/content/drive/MyDrive/remplacement/dicts/dico-s.tsv
/content/drive/MyDrive/remplacement/dicts/dico-r.tsv
/content/drive/MyDrive/remplacement/dicts/dico-q.tsv
/content/drive/MyDrive/remplacement/dicts/dico-p.tsv
/content/drive/MyDrive/remplacement/dicts/dico-m.tsv
/content/drive/MyDrive/remplacement/dicts/dico-d-bis.tsv
/content/drive/MyDrive/remplacement/dicts/

,syn
mot,
ichnographie,plan d'un édifice
ichtyophage,qui ne vit que de poisson
iconographie,description des images
iconologie,"interprétation des images,des édifices"
idiome,langue propre d'une nation
...,...
néréide,nymphe
némésis,revanche
nénette,amante


In [4]:
# 3879 mots et 3844 mots uniques 
len(set(mot_syn.index))

3844

In [5]:
#35 occurences
len(set(mot_syn[mot_syn.index.duplicated()].index))

35

In [6]:
#3843 mots non null donc probablement un seul mot est null
len(set(mot_syn[mot_syn.index.notnull()].index))

3843

In [7]:
#3843 mot et 3838 syno, voir 5 syno na (valeur null) 
mot_syn.describe()

,syn
count,3874
unique,3233
top,?
freq,25


In [8]:
#supprimer la deuxième occurence d'un doublon 
mot_syn = mot_syn[~mot_syn.index.duplicated(keep='first')] 
#supprime les index null
mot_syn = mot_syn[mot_syn.index.notnull()] 

# suppression des syno null et tranpose mot_syn en syn_mot
syn_mot = mot_syn.dropna().transpose()


In [9]:
syn_mot

mot,ichnographie,ichtyophage,iconographie,iconologie,idiome,idolatrer,ignare,ignicole,ignoble,ignominie,illégitimement,illicitement,illuminer,illusion,illustre,imberbe,imbiber,imbroglio,immangeable,immanquablement,immensité,immodérément,immoler,immonde,immuable,imperfection,impérieusement,impéritie,imperturbable,impétueux/ euse,impiété,impitoyable,implorer,importun/ une,imposteur,imposture,imprécation,improuver,improviser,imprudemment,...,nématode,nèfle,nénies,néoménie,nervure,névritique,niable,nielle,nitre,nobiliaire,nocher,noliser,nomie,novale,novateur,novation,novelle,novissimé,nuaison,nubile,nudité,nome,nuptial,notule,nouage,nouille,nougat,nuer,nitescence,nibliau,nocuité,nolisement,niaise,néphélion,nélombo,néréide,némésis,nénette,nèpe,néolisme
syn,plan d'un édifice,qui ne vit que de poisson,description des images,"interprétation des images,des édifices",langue propre d'une nation,vénérer des idoles,illéttré,adorateur du feu,bassesse,grand déshonneur,injustement,contre le droit,éclairer,apparence trompeuse,éclatant,sans barbe,abreuver,confusion,pas bon à manger,infailliblement,grandeur,avec excés,bruler vif,sale,qui ne change pas,défaut,superbement,ignorance dans sa profession,qui ne peut etre troublé,violent,athé,insensible,demander avec humilité,facheux,trompeur,tromperie,malédiction,désapprouver,faire sans préparation,"défaut, risque",...,lombric,"Sorte de fruit qui a plusieurs noyaux, dont la...",Chants funèbres qui se faisoient dans l'ancien...,"Terme d'Astronomie ancienne, tiré du Grec, et...",arête,Il se dit Des médicamens propres aux maladies ...,douteux,brouuillard,Sel formé par l'union de l'acide qu'on nomme N...,Catalogue des maisons nobles,pilote,fréter,règle,Terre nouvellement défrichée et mise en valeur,créateur,changement,"Constitutions de l'Empereur Justinien, qui f...",Tout récemment.,Terme de Marine. Il se dit De tout le temps qu...,fait,anatomie,Chants funèbres qui se faisoient dans l'ancien...,conjugal,note,tissage,bête,gâteau,assortir,clarté,gentillâtre,dangerosité,affrètement,cruche,albugo,lotus,nymphe,revanche,amante,d'eau,nouveau


In [10]:
#verification  
len(set(mot_syn.index)), len(set(mot_syn[mot_syn.index.notnull()].index))

(3843, 3843)

In [11]:
#syn_mot.columns.values.shape

In [12]:
#syn_mot['ichnographie'].iloc[0].split()

In [13]:
dict_mot_syn_uniq={}
# A partir de mot_syn creer un dictio de mots unique (filtre les groupes de mots syno)
for mot_complique in syn_mot.columns.values:
    #print(mot_complique)
    if(len(syn_mot[mot_complique]["syn"].split())==1):
            dict_mot_syn_uniq[mot_complique] = syn_mot[mot_complique]["syn"]

In [14]:
dict_mot_syn_uniq

{'ignare': 'illéttré',
 'ignoble': 'bassesse',
 'illégitimement': 'injustement',
 'illuminer': 'éclairer',
 'illustre': 'éclatant',
 'imbiber': 'abreuver',
 'imbroglio': 'confusion',
 'immanquablement': 'infailliblement',
 'immensité': 'grandeur',
 'immonde': 'sale',
 'imperfection': 'défaut',
 'impérieusement': 'superbement',
 'impétueux/ euse': 'violent',
 'impiété': 'athé',
 'impitoyable': 'insensible',
 'importun/ une': 'facheux',
 'imposteur': 'trompeur',
 'imposture': 'tromperie',
 'imprécation': 'malédiction',
 'improuver': 'désapprouver',
 'impudicité': 'vice',
 'imputer': 'attribuer',
 'inanition': 'faiblesse',
 'incaguer': 'défier',
 'incarcérer': 'emprisonner',
 'incision': 'coupure',
 'inciter': 'pousser ',
 'incliner': 'baisser',
 'incommode': 'facheux',
 'incompétence': 'défaut',
 'incomplexe': 'simple',
 'incrédulité': 'opposition',
 'inculpation': 'accusation',
 'inculper': 'accuser',
 'incuriosité': 'ignorance ',
 'indemniser': 'dédommager',
 'indigent / ente': 'nécess

In [15]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("fr_core_news_sm")
from verbecc import conjugator
cg = conjugator.Conjugator("fr")
from nltk.parse.generate import generate, demo_grammar
from nltk import CFG
import pattern.fr

In [16]:
test_grammar =  """
    S -> NP VP
    NP -> Det N
    PP -> P NP
    VP -> 'est'| 'sera' | 'était' | 'a été' | 'enfournait'|  PP
    Det -> 'le' | 'un'
    N -> 'camphre' | 'concoction' | 'couard' | 'courroux' | 'créance' | 'test' | 'chien'
    P -> 'après' | 'avant' |  'avec'  |  'sans' |  'depuis'
"""
grammar = CFG.fromstring(test_grammar)
print(grammar)
len(list(generate(grammar)))

Grammar with 23 productions (start state = S)
    S -> NP VP
    NP -> Det N
    PP -> P NP
    VP -> 'est'
    VP -> 'sera'
    VP -> 'était'
    VP -> 'a été'
    VP -> 'enfournait'
    VP -> PP
    Det -> 'le'
    Det -> 'un'
    N -> 'camphre'
    N -> 'concoction'
    N -> 'couard'
    N -> 'courroux'
    N -> 'créance'
    N -> 'test'
    N -> 'chien'
    P -> 'après'
    P -> 'avant'
    P -> 'avec'
    P -> 'sans'
    P -> 'depuis'


1050

In [17]:
sentences = list(generate(grammar))
'. '.join(list(map(lambda sentence : ' '.join(sentence), sentences)))

'le camphre est. le camphre sera. le camphre était. le camphre a été. le camphre enfournait. le camphre après le camphre. le camphre après le concoction. le camphre après le couard. le camphre après le courroux. le camphre après le créance. le camphre après le test. le camphre après le chien. le camphre après un camphre. le camphre après un concoction. le camphre après un couard. le camphre après un courroux. le camphre après un créance. le camphre après un test. le camphre après un chien. le camphre avant le camphre. le camphre avant le concoction. le camphre avant le couard. le camphre avant le courroux. le camphre avant le créance. le camphre avant le test. le camphre avant le chien. le camphre avant un camphre. le camphre avant un concoction. le camphre avant un couard. le camphre avant un courroux. le camphre avant un créance. le camphre avant un test. le camphre avant un chien. le camphre avec le camphre. le camphre avec le concoction. le camphre avec le couard. le camphre avec l

In [18]:
def get_verb_conj (infinitive, mood, tense, person) :
  all_conj_form = cg.conjugate(infinitive)
  conj = all_conj_form["moods"][mood][tense][person]
  verb_conj = ''
  j_apostrf = conj.partition("'")[0] + conj.partition("'")[1]
  if j_apostrf in ["j'","J'"] :
    verb_conj = conj.partition("'")[-1]
  else :   
    verb_conj =  conj.partition(' ')[-1]
  return  verb_conj 


def mood_tense_pers (verb_conj,verb_inf) :
  all_conj_form = cg.conjugate(verb_inf)
  info = {}
  for mood  in list(all_conj_form["moods"].keys()) : 
    for tense  in list(all_conj_form["moods"][mood].keys()) :
      for person in range(len (all_conj_form["moods"][mood][tense])) :
        if get_verb_conj (verb_inf,mood,tense,person) == verb_conj :
          info["person"] = person
          info["mood"] = mood
          info["tense"] = tense
          break
  return info

def conjugate_verb (verb_synonyme, prev_token, token) : 
  info = {}
  if prev_token.lemma_ in ['avoir','être'] :
    verb_complet = prev_token.text + ' ' + token.text
    info = mood_tense_pers (verb_complet, token.lemma_)                    
    replacing_verb = cg.conjugate(verb_synonyme)["moods"][info["mood"]][info["tense"]][info["person"]]
  else :
    info = mood_tense_pers (token.text, token.lemma_)
    replacing_verb = cg.conjugate(verb_synonyme)["moods"][info["mood"]][info["tense"]][info["person"]]
  return replacing_verb.partition(' ')[-1]   


In [19]:
#renvoie le pluriel d'un mot
def to_plural (word) :
  return pattern.fr.pluralize(word)

# Corriger les mots mal orthographiés
def aut_word_correction (word) :
  # la liste des mots probablement correctes
  list_word = pattern.fr.suggest(word)
  index = 0
  max_value = 0
  # on recupère le mot le plus probable
  for i in range(len(list_word)) : 
    if list_word[i][1] > max_value : 
        index = i 
        max_value = list_word[i][1] 
  word_select = list_word[index][0]
  return  word_select

def isSing(doc,position_mot):
    #determine si le mot matché est au singulier ou au pluriel à partir de sa morphologie (obtenu via spacy)
    #morph est un string au format "Feat1=Val1|Feat2=Val2|...", par exemple "Gender=Fem|Number=Sing"
    token =  doc[position_mot]
    morph_word = token.morph
    if "Number=Sing" in morph_word :
        return True
    elif "Number=Plur" in morph_word :
        return False
    #si on ne peut pas le déterminer à partir de la morphology du mot    
    if (position_mot > 0):
        
        #on essaie de le déterminer à partir de la morphology du mot qui précède si les 2 sont liés (https://spacy.io/usage/linguistic-features#navigating)
        if doc[position_mot-1].head.text == token.text:
            morph_previous = doc[position_mot -1].morph # morph2 = doc[position_mot -1].morph ?
            if "Number=Sing" in morph_previous:
                return True
            elif "Number=Plur" in morph_previous:
                return False
    return "Not difine"


def isFem(doc,position_mot):
    token =  doc[position_mot]
    morph_word = token.morph
    if "Gender=Fem" in morph_word:
        return True
    elif "Gender=Masc" in morph_word:
        return False
         
    if (position_mot > 0):
        if doc[position_mot-1].head.text == token.text:
            morph_previous = doc[position_mot -1].morph
            if "Gender=Fem" in morph_previous:
                return True
            elif "Gender=Masc" in morph_previous:
                return False
       
    return "Not difine"



In [20]:
def replace (syn_mot , initial_text) :

    # Initialise le Matcher avec le vocab partagé  
    matcher = Matcher(nlp.vocab)
    #texte à renvoyer
    text = "" 
    #position de la fin du dernier matching
    last_end = 0 

    #Itère sur les mots compliqués
    for mot_complique in syn_mot.columns.values:
        matcher.add(mot_complique, [[{"LEMMA": mot_complique}]]) 
        
    # Traite le text initial    
    doc = nlp(initial_text)
    # Appelle le matcher sur le doc
    matches = matcher(doc)
    
    # etres sur que le text contient des mots compliqués présentent dans notre base
    if len(matches)>0 :
        print(f"les mots à remplacer dans ce text sont : {[doc[start:end] for match_id, start, end in matches ]}")

        # Itère sur les correspondances
        for match_id, start, end in matches:
            matched_span = doc[start:end] 
            mot_complique = doc.vocab.strings[match_id] 
            
            # le synonym est un et un seule mot
            if (len(syn_mot[mot_complique]["syn"].split())==1):              
                remplacement_word = syn_mot[mot_complique]["syn"] 
                remplacement_word = aut_word_correction (remplacement_word) 

                #si le mot matché n'est pas un verbe
                if not doc[start].pos_ == "VERB":

                    #teste si le mot est feminin 
                    #if isFem(doc,start) :
                        #pour le moment, on détecte si le mot est au féminin mais on ne fait rien av   

                    # teste si c'est le mot est pluriel    
                    if not isSing(doc,start) :                 
                        remplacement_word = to_plural(remplacement_word)

                # Si le mot est un verbe        
                else:  
                    #si le verbe est à l'infinitif             
                    if doc[start].text == doc[start].lemma_ or doc[start].text == doc[start].lemma_.capitalize() :
                        remplacement_word = remplacement_word
                    #si le verbe est conjugué    
                    else :

                        prev_token = None
                        prev_token_text = ""

                        #si le verbe n'est en début de texte 
                        if start > 0:
                            prev_token = doc[start-1]
                            prev_token_text = doc[start-1].text
                            #le verbe est à l'infinitif passé
                            if prev_token.text in ["Avoir","avoir","être", "Etre"] :
                                remplacement_word = cg.conjugate(remplacement_word)["moods"]["participe"]["participe-passé"][0]
                            else :                           
                                remplacement_word = conjugate_verb(remplacement_word, prev_token, doc[start])                                
                                  
                # si le mot est en début de texte 
                if start == 0 :               
                    text = text  + remplacement_word.capitalize()

                # si le mot est en début de phrase    
                elif doc[start-1].pos_ == "PUNCT" and end < len(doc): 
                    text = text + doc[last_end:start].text + " "  + remplacement_word.capitalize()

                # le mot est en milieu de phrase    
                elif len(remplacement_word.split())==1 :  
                    text = text + doc[last_end:start].text + " "  + remplacement_word
                else : 
                    text = text + doc[last_end:start-1].text + " "  + remplacement_word
                    
                #l'insertion d'un mot doit etre suivit de espace avant de passer à la prochaine insertion   
                if end < len(doc) and not doc[end].pos_ == "PUNCT":
                    text = text + " " 

                last_end = end

            #si le synonym est un group de mots
            else : 
                remplacement_word = syn_mot[mot_complique]["syn"] 
                text = text + doc[last_end:start+1].text + f"({remplacement_word})"
                last_end = end
                
        
        # si le text n'est pas encore terminer aprés avoir traiter tous les mots compliqués   
        if last_end < len(doc):
            text = text + doc[last_end:(len(doc))].text
   
    #Pas de mots compliqués dans la base 
    else :
      print(f"pas de mots compliqués dans le text")
      text = initial_text  
    return text




####replace_test error detection

####test

In [21]:
#recupération du text
get_text = '. '.join(list(map(lambda sentence : ' '.join(sentence), sentences)))
get_text

'le camphre est. le camphre sera. le camphre était. le camphre a été. le camphre enfournait. le camphre après le camphre. le camphre après le concoction. le camphre après le couard. le camphre après le courroux. le camphre après le créance. le camphre après le test. le camphre après le chien. le camphre après un camphre. le camphre après un concoction. le camphre après un couard. le camphre après un courroux. le camphre après un créance. le camphre après un test. le camphre après un chien. le camphre avant le camphre. le camphre avant le concoction. le camphre avant le couard. le camphre avant le courroux. le camphre avant le créance. le camphre avant le test. le camphre avant le chien. le camphre avant un camphre. le camphre avant un concoction. le camphre avant un couard. le camphre avant un courroux. le camphre avant un créance. le camphre avant un test. le camphre avant un chien. le camphre avec le camphre. le camphre avec le concoction. le camphre avec le couard. le camphre avec l

In [23]:
test1 = """Concoction texte est une série orale ou écrite de mots perçus comme constituant un ensemble cohérent.
        Le camphre avait enfourné. Le camphre dit avoir enfourner. Enfourner c'est pour enfourner. Nous enfournons. Concoction de sens et utilisant les structures propres à une langue. 
        Un texte n'a pas de longueur déterminée sauf dans le cas de poèmes à forme fixe comme le sonnet ou le haïku.
        Cependant qu'appelle t-on la poésie des néolismes?"""
test2 = "Avoir enfourner le camphre. Courroux après un camphre. le camphre avant un créance.le camphre avec le couard."    
print(replace(syn_mot , test1), sep='.')


les mots à remplacer dans ce text sont : [Concoction, enfourné, enfourner, Enfourner, enfourner, enfournons, Concoction, structures, déterminée, néolismes]
Direction texte est une série orale ou écrite de mots perçus comme constituant un ensemble cohérent.
        Le camphre avait commencé. Le camphre dit avoir commencer. Commencer c'est pour commencer. Nous commençons. Direction de sens et utilisant les dispositions propres à une langue. 
        Un texte n'a pas de longueur déterminée(fixer des limites)sauf dans le cas de poèmes à forme fixe comme le sonnet ou le haïku.
        Cependant qu'appelle t-on la poésie des nouveaux?


####systeme de recuperation automatique de text 

In [ ]:
from joblib import Memory
from pathlib import Path
import requests
import fitz

path = Path('.')
CACHE_DIR =  path / '.jupyter_cache'
memory = Memory(CACHE_DIR, verbose=0)

@memory.cache
def download(url, dst):
    response = requests.get(url, allow_redirects=True)
    with open(dst, 'wb') as f:
        f.write(response.content)

# extraction de text sur doc pdf en ligne
def get_text (url ,page_no , path_name) :
  download(url, path_name)
  doc = fitz.open(path_name)
  ponct=['.','!','?',',',';', ':']
  page = doc[page_no]
  opt = 'text'
  text = page.getText(opt)
  text = text.splitlines()
  tx=''
  for sent in text : 
    if sent[-1] in ponct : 
      tx = tx + ' ' + sent
    else :
      tx = tx + '.' + ' ' + sent  
  return tx        

In [ ]:
url_falc = 'http://www.santetresfacile.fr/files/regles_du_facile_a_lire_et_a_comprendre.pdf'
fn = path / 'falc.pdf'
page_no = 0
text = get_text(url_falc,page_no, fn)
replace_test (syn_mot,text) 

    

In [ ]:
text

In [ ]:
replace_test (syn_mot,text)

####Replace without IA

In [ ]:
import re
def replace(words,text):
        #remplacement des mots tel quel sans IA, n'est pas utilisé dans le cadre des mots compliqués
        keys = words.keys()
        for i in keys:
            print(i)
            pattern = re.compile(r'\b'+i+r'\b', re.I)
            #compile un motif vers une expression rationnelle compilée
            #qu'on utilise ensuite pour faire la recherche dans le texte
            #re.I ignorer la casse
            text = re.sub(pattern, words[i], text)
            #Renvoie la chaîne obtenue en remplaçant les occurrences
            #de pattern dans string par le remplacement words[i].
        return text

In [ ]:
replace(dict,text)

In [ ]:
dict